# DataRobot API Starter Guide

### Configure DataRobot env

In [1]:
import datarobot as dr
dr.Client(config_path = "./drconfig.yaml")

###  Create a Dataset object and a Project

In [2]:
# Set the location of the dataset
training_dataset_file_path = './datasets/auto-mpg.csv'
test_dataset_file_path = './datasets/auto-mpg-test.csv'

# Load the dataset
train_dataset = dr.Dataset.create_from_file(training_dataset_file_path)

# Create a new project based on the dataset
project = dr.Project.create_from_dataset(train_dataset.id, project_name="Auto MPG DR-Client")

### Training models using AutoPilot

In [3]:
from datarobot import AUTOPILOT_MODE

# Set the project's target and initiate Autopilot (runs in Quick mode unless a different mode is specified)
project.analyze_and_model(target='mpg', worker_count=-1, mode=AUTOPILOT_MODE.QUICK)

# Open the project's Leaderboard to monitor the progress in UI.
project.open_in_browser()

# Wait for the model creation to finish
project.wait_for_autopilot()

model = project.get_top_model()

In progress: 2, queued: 6 (waited: 0s)
In progress: 2, queued: 6 (waited: 1s)
In progress: 2, queued: 6 (waited: 3s)
In progress: 2, queued: 6 (waited: 5s)
In progress: 2, queued: 6 (waited: 7s)
In progress: 3, queued: 5 (waited: 9s)
In progress: 3, queued: 5 (waited: 14s)
In progress: 3, queued: 4 (waited: 21s)
In progress: 2, queued: 4 (waited: 35s)
In progress: 3, queued: 1 (waited: 57s)
In progress: 4, queued: 0 (waited: 78s)
In progress: 0, queued: 16 (waited: 99s)
In progress: 0, queued: 14 (waited: 120s)
In progress: 2, queued: 10 (waited: 141s)
In progress: 3, queued: 7 (waited: 163s)
In progress: 3, queued: 4 (waited: 184s)
In progress: 2, queued: 2 (waited: 205s)
In progress: 1, queued: 0 (waited: 227s)
In progress: 0, queued: 0 (waited: 248s)
In progress: 0, queued: 0 (waited: 269s)
In progress: 0, queued: 0 (waited: 290s)
In progress: 1, queued: 0 (waited: 311s)
In progress: 0, queued: 0 (waited: 333s)
In progress: 0, queued: 0 (waited: 354s)
In progress: 0, queued: 0 (wait

c:\Users\ROG\AppData\Local\Programs\Python\Python312\Lib\site-packages\datarobot\models\project.py:2176: DataRobotDeprecationWarning: `get_models` has been marked for change in `3.4`, will be changed in `4.0`. In 4.0, the flag `use_new_models_retrieval` will be always `True`.
         Fewer attributes will be returned in the response, see the docstring for more details. 
  models = self.get_models()


### Predicting data for the test set

In [4]:
# Test predictions on new data
prediction_data = project.upload_dataset(test_dataset_file_path)  # Additional code to upload test data
predict_job = model.request_predictions(prediction_data.id)  # Directly add test data here instead of the ID of the uploaded dataset
predictions = predict_job.get_result_when_complete()
predictions.head()

,row_id,prediction
0,0,29.752946
1,1,29.752946


In [5]:
project.get_blueprints()

[Blueprint(Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss)),
 Blueprint(Nystroem Kernel SVM Regressor),
 Blueprint(eXtreme Gradient Boosted Trees Regressor (Tweedie Loss: 1.5)),
 Blueprint(Keras Slim Residual Neural Network Regressor using Adaptive Training Schedule (1 Layer: 64 Units)),
 Blueprint(Nystroem Kernel SVM Regressor (Tweedie Deviance)),
 Blueprint(Eureqa Regressor (Instant Search: 40 Generations)),
 Blueprint(Lasso Regressor),
 Blueprint(Ridge Regressor),
 Blueprint(Nystroem Kernel SVM Regressor (Gamma Deviance)),
 Blueprint(Elastic-Net Regressor (L2 / Poisson Deviance)),
 Blueprint(Keras Residual AutoInt Regressor using Training Schedule (2 Attention Layers with 2 Heads, 2 Layers: 96, 96 Units)),
 Blueprint(Keras Residual Cross Network Regressor using Training Schedule (3 Cross Layers, 4 Layers: 96, 96, 72, 72 Units)),
 Blueprint(Linear Regression),
 Blueprint(Gradient Boosted Greedy Trees Regressor (Least-Squares Loss)),
 Blueprint(Gradient Boosted Trees Reg

### Evaluating metrics for the model

### Deploying the model

In [6]:
# Retrieve prediction server id before deploying the model
prediction_servers = dr.Deployment.list()

deployment = dr.Deployment.create_from_learning_model(
    model_id = model.id, label="MPG Prediction Server",
    description = "Deployed with DataRobot client",
    default_prediction_server = prediction_servers[1].id
)

# View the deployment status
service_stats = deployment.get_service_stats()
print(service_stats.metrics)

IndexError: list index out of range